In [1]:
import os
import sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
sys.path.append('/Users/matangrinberg/Library/CloudStorage/GoogleDrive-matan.grinberg@gmail.com/My Drive/(21-24) University of California, Berkeley/ML HEP/parametrized-classifiers/data')

In [2]:
import pythia8
import fastjet
import numpy as np
import numpy.random as random

In [3]:
def generate_event(event_generator, jet_definition, alphaS, aLund, probStoUD):
    boolisphoton = True
    while (boolisphoton):
        event_generator.next()
        boolisphoton = False
        particlesForJets = []
        for i in range(event_generator.event.size()):
            p = fastjet.PseudoJet(event_generator.event[i].px(), event_generator.event[i].py(), event_generator.event[i].pz(), event_generator.event[i].e())
            p.set_user_index(i)
            if (event_generator.event[i].isFinal()==False):
                continue;
            if (abs(event_generator.event[i].id())==12):
                continue;
            if (abs(event_generator.event[i].id())==14):
                continue;
            if (abs(event_generator.event[i].id())==13):
                continue;
            if (abs(event_generator.event[i].id())==16):
                continue;
            particlesForJets += [p]
            pass

        cs = fastjet.ClusterSequence(particlesForJets, jet_definition)
        myJets = fastjet.sorted_by_pt(cs.inclusive_jets(25.0));

        if (len(myJets) > 0):
            if (len(myJets[0].constituents())==1):
                myid = myJets[0].constituents()[0].user_index()
                origin = 0
                while (event_generator.event[myid].id()==22):
                    myid = event_generator.event[myid].mother1()
                    origin = event_generator.event[myid].id()
                    pass
                if (abs(origin)==11):
                    boolisphoton = True
                    pass
                pass
            pass

        if (len(myJets)==0):
            boolisphoton = True
        else:
            outstring = np.zeros((51,7))
            for i in range(len(myJets[0].constituents())):
                outstring[i] = myJets[0].constituents()[i].px(), myJets[0].constituents()[i].py(), myJets[0].constituents()[i].pz(), myJets[0].constituents()[i].user_index(), alphaS, aLund, probStoUD
                pass
    return outstring

In [4]:
# When t=0 we want (0.16, 0.7, 0.27)
def parameter_distribution(size, t, max_fineness=100000):
    alphaS0, aLund0, probStoUD0 = np.array([0.16]), np.array([0.7]), np.array([0.27])
    
    if t==0:
        return alphaS0, aLund0, probStoUD0

    width_alphaS, width_aLund, width_probStoUD = 0.04, 0.1, 0.05

    alphaS = alphaS0 + width_alphaS * random.randint(low = - max_fineness * t, high = max_fineness * t, size=size) / (max_fineness * t)
    aLund = aLund0 + width_aLund * random.randint(low = - max_fineness * t, high = max_fineness * t, size=size) / (max_fineness * t)
    probStoUD = probStoUD0 + width_probStoUD * random.randint(low = - max_fineness * t, high = max_fineness * t, size=size) / (max_fineness * t)
    
    return alphaS, aLund, probStoUD

In [5]:
def generate_dataset(num_rand, num_mult, t):
    # alphaS = random.uniform(low=0.1, high=0.2, size=(num_rand))
    # aLund = random.uniform(low=0.2, high=0.3, size=(num_rand))
    # probStoUD = random.uniform(low=0.6, high=0.8, size=(num_rand))
    
    alphaS, aLund, probStoUD = parameter_distribution(num_rand, t)
    n = num_rand * num_mult
    dataset = np.zeros((n, 51, 7))
    
    for i in range(num_rand):
        print(i)
        alphaS_i = alphaS[i]
        aLund_i = aLund[i]
        probStoUD_i = probStoUD[i]
        evgen = pythia8.Pythia()
        jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.8)

        evgen.readString("WeakSingleBoson:ffbar2gmZ  = on");
        evgen.readString("23:onMode = off");
        evgen.readString("23:onIfAny =  1 2 3");
        evgen.readString("Print:quiet = on");
        evgen.readString("Beams:idA = 11");
        evgen.readString("Beams:idB = -11");
        evgen.readString("Beams:eCM = 92");

        evgen.readString("TimeShower:alphaSvalue = " + str(alphaS_i))
        evgen.readString("StringZ:aLund = " + str(aLund_i))
        evgen.readString("StringFlav:probStoUD = " + str(probStoUD_i))
        evgen.init()
        
        for j in range(num_mult):
            dataset[i * num_mult + j] = generate_event(evgen, jetdef, alphaS_i, aLund_i, probStoUD_i)
            
    return dataset

In [6]:
def generate_dataset_ref(num_rand, num_mult, t):
    n = num_rand * num_mult
    dataset = np.zeros((n, 51, 7))
    
    evgen = pythia8.Pythia()
    jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.8)

    evgen.readString("WeakSingleBoson:ffbar2gmZ  = on");
    evgen.readString("23:onMode = off");
    evgen.readString("23:onIfAny =  1 2 3");
    evgen.readString("Print:quiet = on");
    evgen.readString("Beams:idA = 11");
    evgen.readString("Beams:idB = -11");
    evgen.readString("Beams:eCM = 92");

    evgen.readString("TimeShower:alphaSvalue = 0.16")
    evgen.readString("StringZ:aLund = 0.7")
    evgen.readString("StringFlav:probStoUD = 0.27")
    evgen.init()

    alphaS_fake, aLund_fake, probStoUD_fake = parameter_distribution(n, t)
    
    for i in range(n):
        dataset[i] = generate_event(evgen, jetdef, alphaS_fake[i], aLund_fake[i], probStoUD_fake[i])
            
    return dataset

In [ ]:
num_random = 10000
num_mult = 10
t = 0.001

x0 = generate_dataset_ref(num_random, num_mult, t)
x1 = generate_dataset(num_random, num_mult, t)

y0 = np.zeros(num_random * num_mult)
y1 = np.ones(num_random * num_mult)

In [ ]:
data_dir = '/global/home/users/mgrinberg/parametrized-classifiers/data/'
run_name = 'interpolate_standard'
run_name += '_' + 'n' + str(num_random*num_mult) + 't' + str(t)

In [ ]:
x = np.concatenate((x0, x1), axis=0)
y = np.concatenate((y0, y1), axis=0)

In [ ]:
np.savez(data_dir + run_name, x, y)

In [ ]:
data = np.load(data_dir + run_name + '.npz')

In [ ]:
data['arr_0'].shape